# TMS-OS (Tiered Multi-Sensor: Optical & SAR)

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.interpolate import interp1d
from scipy.stats import sigmaclip
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

# 1. Exploring Sentinel-2 time-series

In [ ]:
# read in sentinel-2 data
s2 = pd.read_csv("../data/s2-sirindhorn.csv", parse_dates=['date']).set_index('date')
s2.head()

In [ ]:
# calculate cloud cover in % over the ROI
s2['cloud_percentage'] = (s2['cloud_area']*100)/(s2['water_area_clustering']+s2['non_water_area_clustering']+s2['cloud_area'])
s2.head()

Plot the data

In [ ]:
fig = make_subplots(
    rows=2,
    row_heights=[0.8, 0.2],
    shared_xaxes=True,
    vertical_spacing = 0.05
)

# Raw Surface Area
fig.add_trace(go.Scatter(
    x = s2.index,
    y = s2['water_area_clustering'],
    name = 'Raw Reservoir Surface Area',
    mode = 'lines+markers'
), row=1, col=1)

# Cloud Corrected Surface Area
fig.add_trace(go.Scatter(
    x = s2.index,
    y = s2['water_area'],
    name = 'Cloud-Corrected Reservoir Surface Area',
    mode = 'lines+markers'
), row=1, col=1)

# Cloud
fig.add_trace(go.Bar(
    x = s2.index,
    y = s2['cloud_percentage'],
    name = 'Cloud Cover (%)',
    marker = dict(
        color = 'red',
    )
), row=2, col=1)

fig.update_layout(
    title=dict(                                         # title
        text='Reservoir Surface Areas - Sentinel-2',
        xanchor='center',
        x=0.5
    ),
    legend=dict(                                        # legend
        orientation = 'h',
        yanchor='top',
        y=-0.08,
        xanchor='right',
        x=1.0,
        bordercolor="grey",
        borderwidth=1
    ),
    margin=dict(l=20, r=20, t=60, b=20)                 # margins
)

fig

_There are a lot of `-1` values, what are those??_
- When the cloud cover is >90% the script returns `-1` as a fill value. Essentially, we don't have a data point there.

Let's remove these values

In [ ]:
# set all the -1 values as np.nan (Not-A-Number)
s2.loc[s2['cloud_percentage']>90, ['water_area_clustering', 'non_water_area_clustering', 'cloud_area', 'water_area']] = np.nan

# drop all the np.nan values
s2.dropna(inplace=True)

s2.head()

Plot the data again

In [ ]:
fig = make_subplots(
    rows=2,
    row_heights=[0.8, 0.2],
    shared_xaxes=True,
    vertical_spacing = 0.05
)

# Raw Surface Area
fig.add_trace(go.Scatter(
    x = s2.index,
    y = s2['water_area_clustering'],
    name = 'Raw Reservoir Surface Area',
    mode = 'lines+markers'
), row=1, col=1)

# Cloud Corrected Surface Area
fig.add_trace(go.Scatter(
    x = s2.index,
    y = s2['water_area'],
    name = 'Cloud-Corrected Reservoir Surface Area',
    mode = 'lines+markers'
), row=1, col=1)

# Cloud
fig.add_trace(go.Bar(
    x = s2.index,
    y = s2['cloud_percentage'],
    name = 'Cloud Cover (%)',
    marker = dict(
        color = 'red',
    )
), row=2, col=1)

fig.update_layout(
    title=dict(                                         # title
        text='Reservoir Surface Areas - Sentinel-2',
        xanchor='center',
        x=0.5
    ),
    legend=dict(                                        # legend
        orientation = 'h',
        yanchor='top',
        y=-0.08,
        xanchor='right',
        x=1.0,
        bordercolor="grey",
        borderwidth=1
    ),
    margin=dict(l=20, r=20, t=60, b=20)                 # margins
)

fig

Notice the sudden drops in surface areas, which usually occur during high cloud cover conditions, but may also happen during cloud-free days. These sudden drops (> 100 sq. km.) followed by sudden rise of similar magnitude in a span of 5-10 days aren't representative of the true behavior of the reservoirs - these are not actual signals. 

These artifacts can occur due to the automatic nature of the clustering algorithm ([Cascade simple K-Means clustering](https://developers.google.com/earth-engine/apidocs/ee-clusterer-wekacascadekmeans)). In the K-Means clustering algorithm, you'd have to specify the number of clusters ($K$) to form, before performing the clustering. Specifying a hard-coded value for $K$ can (1) be difficult, and (2) create additional issues, especially when several reservoirs are to be mapped at once (such as RAT-Mekong). Moreover, in case of a highly dynamic reservoir, where the area may change drastically during dry and wet seasons, the number of distinct features that can appear as a "cluster" may vary as well. Due to such reasons, it is recommended to use an automatic scheme of choosing the value of $K$, which, in this case, is done using [the Calinski-Harabasz criterion](https://www.tandfonline.com/doi/abs/10.1080/03610927408827101).

The limitation with this method, however, is that the Calinski-Harabasz crierion can sometimes choose a value of $K$ where the cluster representing water pixels gets divided into multiple clusters, during challenging scenarios. Such challenging scenarios can occur due to unmasked cloud cover, sediment-laden the water, intermittent vegetation, and other artifacts of processing done by the Satellite data provider.

**These erroneous values get corrected in the subsequent steps of TMS-OS**

# Let's load the datasets and plot them together

In [ ]:
CLOUD_THRESHOLD = 90
L8_TEMPORAL_RESOLUTION = 16
S2_TEMPORAL_RESOLUTION = 5
S1_TEMPORAL_RESOLUTION = 12

# LANDSAT - 8
l8df = pd.read_csv('../data/l8-sirindhorn.csv', parse_dates=['mosaic_enddate']).rename({
            'mosaic_enddate': 'date',
            'water_area_cordeiro': 'water_area_uncorrected',
            'non_water_area_cordeiro': 'non_water_area', 
            'corrected_area_cordeiro': 'water_area_corrected'
            }, axis=1).set_index('date')
l8df = l8df[['water_area_uncorrected', 'non_water_area', 'cloud_area', 'water_area_corrected']]
l8df['cloud_percent'] = l8df['cloud_area']*100/(l8df['water_area_uncorrected']+l8df['non_water_area']+l8df['cloud_area'])
l8df.replace(-1, np.nan, inplace=True)

# QUALITY_DESCRIPTION
#   0: Good, not interpolated either due to missing data or high clouds
#   1: Poor, interpolated either due to high clouds
#   2: Poor, interpolated either due to missing data
l8df.loc[:, "QUALITY_DESCRIPTION"] = 0
l8df.loc[l8df['cloud_percent']>=CLOUD_THRESHOLD, ("water_area_uncorrected", "non_water_area", "water_area_corrected")] = np.nan
l8df.loc[l8df['cloud_percent']>=CLOUD_THRESHOLD, "QUALITY_DESCRIPTION"] = 1

# in some cases l8df may have duplicated rows (with same values) that have to be removed
if l8df.index.duplicated().sum() > 0:
    print("Duplicated labels, deleting")
    l8df = l8df[~l8df.index.duplicated(keep='last')]

# Fill in the gaps in l8df created due to high cloud cover with np.nan values
l8df_interpolated = l8df.reindex(pd.date_range(l8df.index[0], l8df.index[-1], freq=f'{L8_TEMPORAL_RESOLUTION}D'))
l8df_interpolated.loc[np.isnan(l8df_interpolated["QUALITY_DESCRIPTION"]), "QUALITY_DESCRIPTION"] = 2
l8df_interpolated.loc[np.isnan(l8df_interpolated['cloud_area']), 'cloud_area'] = max(l8df['cloud_area'])
l8df_interpolated.loc[np.isnan(l8df_interpolated['cloud_percent']), 'cloud_percent'] = 100
l8df_interpolated.loc[np.isnan(l8df_interpolated['non_water_area']), 'non_water_area'] = 0
l8df_interpolated.loc[np.isnan(l8df_interpolated['water_area_uncorrected']), 'water_area_uncorrected'] = 0

# Interpolate bad data
l8df_interpolated.loc[:, "water_area_corrected"] = l8df_interpolated.loc[:, "water_area_corrected"].interpolate(method="linear", limit_direction="forward")


# Read in Sentinel-2 data
s2df = pd.read_csv('../data/s2-sirindhorn.csv', parse_dates=['date']).set_index('date')
s2df = s2df[['water_area_clustering', 'non_water_area_clustering', 'cloud_area', 'water_area']]
s2df['cloud_percent'] = s2df['cloud_area']*100/(s2df['water_area_clustering']+s2df['non_water_area_clustering']+s2df['cloud_area'])
s2df.replace(-1, np.nan, inplace=True)
s2df.loc[s2df['cloud_percent']>=CLOUD_THRESHOLD, ("water_area_clustering", "non_water_area_clustering", "water_area")] = np.nan

# QUALITY_DESCRIPTION
#   0: Good, not interpolated either due to missing data or high clouds
#   1: Poor, interpolated either due to high clouds
#   2: Poor, interpolated either due to missing data
s2df.loc[:, "QUALITY_DESCRIPTION"] = 0
s2df.loc[s2df['cloud_percent']>=CLOUD_THRESHOLD, "QUALITY_DESCRIPTION"] = 1

# in some cases s2df may have duplicated rows (with same values) that have to be removed
if s2df.index.duplicated().sum() > 0:
    print("Duplicated labels, deleting")
    s2df = s2df[~s2df.index.duplicated(keep='last')]

# Fill in the gaps in s2df created due to high cloud cover with np.nan values
s2df_interpolated = s2df.reindex(pd.date_range(s2df.index[0], s2df.index[-1], freq=f'{S2_TEMPORAL_RESOLUTION}D'))
s2df_interpolated.loc[np.isnan(s2df_interpolated["QUALITY_DESCRIPTION"]), "QUALITY_DESCRIPTION"] = 2
s2df_interpolated.loc[np.isnan(s2df_interpolated['cloud_area']), 'cloud_area'] = max(s2df['cloud_area'])
s2df_interpolated.loc[np.isnan(s2df_interpolated['cloud_percent']), 'cloud_percent'] = 100
s2df_interpolated.loc[np.isnan(s2df_interpolated['non_water_area_clustering']), 'non_water_area_clustering'] = 0
s2df_interpolated.loc[np.isnan(s2df_interpolated['water_area_clustering']), 'water_area_clustering'] = 0

# Interpolate bad data
s2df_interpolated.loc[:, "water_area"] = s2df_interpolated.loc[:, "water_area"].interpolate(method="linear", limit_direction="forward")


# Sentinel - 1
# Read in Sentinel-1 data
sar = pd.read_csv('../data/sar-sirindhorn.csv', parse_dates=['time']).rename({'time': 'date'}, axis=1)
sar['date'] = sar['date'].apply(lambda d: np.datetime64(d.strftime('%Y-%m-%d')))
sar.set_index('date', inplace=True)
sar.sort_index(inplace=True)
sar = sar.loc['2019-01-01':, :]
# in some cases sar may have duplicated rows (with same values) that have to be removed
if sar.index.duplicated().sum() > 0:
    sar = sar[~sar.index.duplicated(keep='last')]

# Merging, Filtering and Correcting based on 

In [ ]:
# Defining the functions
# https://gist.github.com/pritamd47/e7ddc49f25ae7f1b06c201f0a8b98348
# Clip time-series
def clip_ts(*tss, which='left'):
    """Clips multiple time-series to align them temporally

    Args:
        which (str, optional): Defines which direction the clipping will be performed. 
                               'left' will clip the time-series only on the left side of the 
                               unaligned time-serieses, and leave the right-side untouched, and 
                               _vice versa_. Defaults to 'left'. Options can be: 'left', 'right' 
                               or 'both'

    Returns:
        lists: returns the time-series as an unpacked list in the same order that they were passed
    """
    mint = max([min(ts.index) for ts in tss])
    maxt = min([max(ts.index) for ts in tss])

    if which == 'both':
        clipped_tss = [ts.loc[(ts.index>=mint)&(ts.index<=maxt)] for ts in tss]
    elif which == 'left':
        clipped_tss = [ts.loc[ts.index>=mint] for ts in tss]
    elif which == 'right':
        clipped_tss = [ts.loc[ts.index<=maxt] for ts in tss]
    else:
        raise Exception(f'Unknown option passed: {which}, expected "left", "right" or "both"./')

    return clipped_tss

def deviation_from_sar(optical_areas, sar_areas, DEVIATION_THRESHOLD = 20, LOW_STD_LIM=2, HIGH_STD_LIM=2):
    """Filter out points based on deviations from SAR reported areas after correcting for bias in SAR water areas. Remove NaNs beforehand.

    Args:
        optical_areas (pd.Series): Time-series of areas obtained using an optical sensor (S2, L8, etc) on which the filtering will be applied. Must have `pd.DatetimeIndex` and corresponding areas in a column named `area`.
        sar_areas (pd.Series): Time-series of S1 surface areas. Must have `pd.DatetimeIndex` and corresponding areas in a column named `area`.
        DEVIATION_THRESHOLD (number): (Default: 20 [sq. km.]) Theshold of deviation from bias corrected SAR reported 
        LOW_STD_LIM (number): (Default: 2) Lower limit of standard deviations to use for clipping the deviations, required for calculating the bias.
        HIGH_STD_LIM (number): (Default: 2) Upper limit of standard deviations to use for clipping the deviations, required for calculating the bias.
    """
    # convert to dataframes under the hood
    optical_areas = optical_areas.to_frame()
    sar_areas = sar_areas.to_frame()

    xs = sar_areas.index.view(np.int64)//10**9  # Convert datetime to seconds from epoch
    ys = sar_areas['area']
    sar_area_func = interp1d(xs, ys, bounds_error=False)
    
    # Interpolate and calculate the sar reported areas according to the optical sensor's observation dates
    sar_sarea_interpolated = sar_area_func(optical_areas.index.view(np.int64)//10**9)
    deviations = optical_areas['area'] - sar_sarea_interpolated

    clipped = sigmaclip(deviations.dropna(), low=LOW_STD_LIM, high=HIGH_STD_LIM)
    bias = np.median(clipped.clipped)

    optical_areas['normalized_dev'] = deviations - bias
    optical_areas['flagged'] = False
    optical_areas.loc[np.abs(optical_areas['normalized_dev']) > DEVIATION_THRESHOLD, 'flagged'] = True
    optical_areas.loc[optical_areas['flagged'], 'area'] = np.nan

    return optical_areas['area']

# helper functions
def sar_trend(d1, d2, sar):
    subset = sar['area'].resample('1D').interpolate('linear')
    subset = subset.loc[d1:d2]
    if len(subset) == 0:
        trend = np.nan
    else:
        trend = (subset.iloc[-1]-subset.iloc[0])/((np.datetime64(d2)-np.datetime64(d1))/np.timedelta64(1, 'D'))
    return trend

def backcalculate(areas, trends, who_needs_correcting):
    # identify the first reliable point
    unreliable_pts_at_the_beginning = len(who_needs_correcting[:who_needs_correcting.idxmin()])-1
    corrected_areas = [np.nan] * unreliable_pts_at_the_beginning
    corrected_areas.append(areas.iloc[unreliable_pts_at_the_beginning+1])

    # # calculate previous points
    # for area, correction_required, trend in zip(areas[unreliable_pts_at_the_beginning::-1], who_needs_correcting[unreliable_pts_at_the_beginning::-1], trends[unreliable_pts_at_the_beginning::-1]):
    #     print(area, correction_required, trend)
    
    for area, correction_required, trend in zip(areas[unreliable_pts_at_the_beginning+1:], who_needs_correcting[unreliable_pts_at_the_beginning+1:], trends[unreliable_pts_at_the_beginning+1:]):
        if not correction_required:
            corrected_areas.append(area)
        else:
            corrected_area = corrected_areas[-1] + trend
            corrected_areas.append(corrected_area)
    
    return corrected_areas

def deviation_correction(area, DEVIATION_THRESHOLD, AREA_COL_NAME='area'):
    inner_area = area.copy()

    inner_area.loc[:, 'deviation'] = np.abs(inner_area['trend']-inner_area['sar_trend'])

    inner_area.loc[:, 'erroneous'] = inner_area['deviation'] > DEVIATION_THRESHOLD

    inner_area.loc[:, 'corrected_trend'] = inner_area['trend']
    inner_area.loc[inner_area['erroneous'], 'corrected_trend'] = inner_area['sar_trend']

    areas = backcalculate(inner_area[AREA_COL_NAME], inner_area['corrected_trend'], inner_area['erroneous'])
    inner_area[AREA_COL_NAME] = areas

    return inner_area

def sign_based_correction(area, AREA_COL_NAME='corrected_areas_1', TREND_COL_NAME='corrected_trend_1'):
    inner_area = area.copy()
    inner_area['sign_based_correction_reqd'] = (inner_area['trend']<0)&(inner_area['sar_trend']>0)|(inner_area['trend']>0)&(inner_area['sar_trend']<0)
    inner_area.loc[:, 'corrected_trend'] = inner_area[TREND_COL_NAME]
    inner_area.loc[inner_area['sign_based_correction_reqd'], 'corrected_trend'] = inner_area['sar_trend']

    inner_area['area'] = backcalculate(inner_area[AREA_COL_NAME], inner_area['corrected_trend'], inner_area['sign_based_correction_reqd'])

    return inner_area

def filled_by_trend(filtered_area, sar_trend, days_passed) -> pd.Series:
    """Fills in `np.nan` values of optically obtained surface area time series using SAR based time-series.

    Args:
        filtered_area (pd.Series): Optical sensor based surface areas containing `np.nan` values that will be filled in.
        sar_trend (pd.Series): SAR based surface area trends.
        days_passed (pd.Series): Days sicne last observation of optical sensor observed surface areas.

    Returns:
        pd.Series: Filled nan values
    """
    filled = [filtered_area.iloc[0]]
    for i in range(1, len(filtered_area)):
        if np.isnan(filtered_area.iloc[i]):
            a = filled[-1] + sar_trend.iloc[i] * days_passed.iloc[i]
            filled.append(a)
        else:
            filled.append(filtered_area.iloc[i])
    
    return pd.Series(filled, dtype=float, name='filled_area', index=filtered_area.index)

# Trend based correction function
def trend_based_correction(area, sar, AREA_DEVIATION_THRESHOLD=25, TREND_DEVIATION_THRESHOLD = 10):
    """Apply trend based correction on a time-series

    Args:
        area (pd.DataFrame): Pandas dataframe containing date as pd.DatetimeIndex and areas in column named `area`
        sar (pd.DataFrame): Pandas dataframe containing surface area time-series obtained from Sentinel-1 (SAR). Same format as `area`
        AREA_DEVIATION_THRESHOLD (number): (Default: 25) Threshold value of deviation of optically derived areas from SAR derived areas fro filtering.
        TREND_DEVIATION_THRESHOLD (number): (Default: 10) Threshold value of deviation in trend above which the observation is marked as erroneous and the correction step is applied
    """

    area['filtered_area'] = deviation_from_sar(area['area'], sar['area'], AREA_DEVIATION_THRESHOLD)
    area.rename({'area': 'unfiltered_area'}, axis=1, inplace=True)
    # area.rename({'filtered_area': 'area'}, axis=1, inplace=True)
    
    area_filtered = area.dropna(subset=['filtered_area'])

    area_filtered.loc[:, 'days_passed'] = area_filtered.index.to_series().diff().dt.days
    area_filtered.loc[:, 'trend'] = area_filtered['filtered_area'].diff()/area_filtered['days_passed']

    sar, area_filtered = clip_ts(sar, area_filtered, which='left')
    # sometimes the sar time-series has duplicate values which have to be removed
    sar = sar[~sar.index.duplicated(keep='first')]   # https://stackoverflow.com/a/34297689/4091712
    trend_generator = lambda arg: sar_trend(arg.index[0], arg.index[-1], sar)

    area_filtered.loc[:, 'sar_trend'] = area_filtered['filtered_area'].rolling(2, min_periods=0).apply(trend_generator)

    deviation_correction_results = deviation_correction(area_filtered, TREND_DEVIATION_THRESHOLD, AREA_COL_NAME='filtered_area')
    area_filtered['corrected_areas_1'] = deviation_correction_results['filtered_area']
    area_filtered['corrected_trend_1'] = deviation_correction_results['corrected_trend']
    
    area['corrected_areas_1'] = area_filtered['corrected_areas_1']
    area['corrected_trend_1'] = area_filtered['corrected_trend_1']

    area.loc[:, 'sar_trend'] = area['unfiltered_area'].rolling(2, min_periods=0).apply(trend_generator)
    area.loc[:, 'days_passed'] = area.index.to_series().diff().dt.days
    
    area, sar = clip_ts(area, sar, which="left")
    first_non_nan = area['corrected_areas_1'].first_valid_index()
    area = area.loc[first_non_nan:, :]

    # fill na based on trends
    area['filled_area'] = filled_by_trend(area['corrected_areas_1'], area['sar_trend'], area['days_passed'])

    return area

In [ ]:
# combine opticals into one dataframes
l8df_interpolated['sat'] = 'l8'
s2df_interpolated['sat'] = 's2'
optical = pd.concat([l8df_interpolated, s2df_interpolated]).sort_index()
optical = optical.loc[~optical.index.duplicated(keep='last')] # when both s2 and l8 are present, keep s2
optical.rename({'water_area_corrected': 'area'}, axis=1, inplace=True)
sar = sar.rename({'sarea': 'area'}, axis=1)

# We need to define the nominal area of the reservoir. It can be a close approximation
NOMINAL_AREA = 200
AREA_DEVIATION_THRESHOLD = 0.05 * 200   # ~5% of the nominal reservoir area works well (Das et al., 2022)

# Apply the trend based corrections
result = trend_based_correction(optical.copy(), sar.copy(), AREA_DEVIATION_THRESHOLD)

# Save the result
result.reset_index().rename({"index": "date"}, axis=1).to_csv("../data/tms-os-sirindhorn.csv", index=False)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(                               # Optical sensors
    x = optical.index,
    y = optical['water_area'],
    mode = 'markers+lines',
    name = 'Surface Area: Optical',
    opacity=0.7
))

fig.add_trace(go.Scatter(                               # SAR 
    x = sar.index,
    y = sar['area'],
    mode = 'markers+lines',
    name = 'Surface Area: SAR',
    opacity=0.7
))

fig.add_trace(go.Scatter(                               # TMS-OS
    x = result.index,
    y = result['filled_area'],
    mode = 'markers+lines',
    name = 'Surface Area: TMS-OS'
))



fig.update_layout(
    title=dict(                                         # title
        text='Reservoir Surface Areas - TMS-OS',
        xanchor='center',
        x=0.5
    ),
    legend=dict(                                        # legend
        orientation = 'h',
        yanchor='top',
        y=-0.08,
        xanchor='right',
        x=1.0,
        bordercolor="grey",
        borderwidth=1
    ),
    margin=dict(l=20, r=20, t=60, b=20)                 # margins
)

fig